In [1]:
from visualdl import LogReader, LogWriter
import random

In [2]:
logdir = "./tmp"
logger = LogWriter(logdir, sync_cycle=10)

In [4]:
# mark the components with 'train' label.
with logger.mode("train"):
    # create a scalar component called 'scalars/scalar0'
    scalar0 = logger.scalar("scalars/scalar0")

# add some records during DL model running.
for step in range(100):
    scalar0.add_record(step, step + random.random())

RuntimeError: [/build/VisualDL/visualdl/storage/storage.cc:43] 

In [4]:
reader = LogReader(logdir)
with reader.mode('train'):
    records = reader.scalar('scalars/scalar0').records()
print(len(records))

99


In [5]:
with logger.mode("train"):
    scalar1 = logger.scalar("scalars/scalar0")

In [6]:
for i, record in enumerate(records):
    scalar1.add_record(i, record)

In [7]:
for step in range(len(records), len(records) + 100):
    scalar1.add_record(step, step + random.random())